<a href="https://colab.research.google.com/github/alibahmanyar/Activity-recognition-2.0/blob/master/HW1_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spark-nlp==4.4.2
!pip install pyspark==3.3.2

In [ ]:
# Downloading data
!wget -nc --user v0 --password v0v1v2v3 https://v0.bahman.me/hw1_data.zip
!unzip -o hw1_data.zip
!ls data

--2023-11-16 12:44:48--  https://v0.bahman.me/hw1_data.zip
Resolving v0.bahman.me (v0.bahman.me)... 172.67.190.56, 104.21.19.213, 2606:4700:3036::ac43:be38, ...
Connecting to v0.bahman.me (v0.bahman.me)|172.67.190.56|:443... connected.
HTTP request sent, awaiting response... 401 Unauthorized
Authentication selected: Basic realm="Who are you?"
Reusing existing connection to v0.bahman.me:443.
HTTP request sent, awaiting response... 200 OK
Length: 2725687 (2.6M) [application/zip]
Saving to: ‘hw1_data.zip’

hw1_data.zip        100%[===================>]   2.60M  2.72MB/s    in 1.0s    

2023-11-16 12:44:50 (2.72 MB/s) - ‘hw1_data.zip’ saved [2725687/2725687]

Archive:  hw1_data.zip
   creating: data/
  inflating: data/Adventures_of_Sherlock_Holmes.txt  
  inflating: data/Alice_in_Wonderland.txt  
  inflating: data/Jane_Eyre.txt      
  inflating: data/JaneAusten.txt     
  inflating: data/Dracula_bromstoker.txt  
  inflating: data/C3.txt             
  inflating: data/C2.txt             
 

In [ ]:
import sparknlp

# Start Spark Session
spark = sparknlp.start()


# Buld pipeline
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import SentenceDetector, UniversalSentenceEncoder, BertSentenceEmbeddings
from pyspark.ml import Pipeline
import pyspark.sql.functions as F

documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")
sentence = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")
embeddings = UniversalSentenceEncoder.pretrained() \
    .setInputCols(["sentence"]) \
    .setOutputCol("sentence_bert_embeddings")
pipeline = Pipeline(stages=[documentAssembler,
                            sentence,
                            embeddings])

In [ ]:
from pyspark.sql.functions import col
text_df = spark.read.text("./data/Adventures_of_Sherlock_Holmes.txt", wholetext=True)
text_df = text_df.select(col('value').alias('text'))
text_df.show(truncate=400)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                                                                                                                                                                                                                                                            text|
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
text_df = text_df.withColumn("text", F.regexp_replace(text_df.text, r'[\r\n]', ' '))
text_df = text_df.withColumn("text", F.regexp_replace(text_df.text, r' +', ' '))

text_df.show(truncate=400)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                                                                                                                                                                                                                                                            text|
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:



model = pipeline.fit(text_df)
result_bert = model.transform(text_df)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
+------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                              sentence|                                                                                                                              Bert_sentence_embeddings|
+------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
result_bert.printSchema()

root
 |-- text: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- sentence: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true

In [ ]:
result_df = result_bert.select(F.explode(F.arrays_zip(result_bert.sentence.result,
                              result_bert.sentence_bert_embeddings.embeddings)).alias("cols")) \
                              .select(F.expr("cols['0']").alias("sentence"),
                                      F.expr("cols['1']").alias("Bert_sentence_embeddings"))
result_df.show(truncate=150)

In [ ]:
from pyspark.ml.stat import Summarizer
import numpy as np
from pyspark.ml.functions import array_to_vector

summarizer = Summarizer.metrics("mean", "count")

vecs = result_df.select(array_to_vector('Bert_sentence_embeddings').alias('vec'))
mean = np.array(vecs.select(Summarizer.mean(vecs.vec)).collect()[0][0])